# Computing the volume underneath a Pareto frontier
This program computes the volume under a frontier of Pareto-optimal solutions to a multi-criterion optimization model, which provides a measure of the conflict among the objectives.

In [1]:
import pandas as pd
import numpy as np
import time

### Preparation for volume computation algorithm

In [2]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/3d/ClimateChange_None/climateChange_EfficientSolutions_NoneOnly.csv")

In [3]:
# Hard-coded manipulations to tidy-up sols

sols = sols.drop(["SolutionIndex","Frontier"],axis=1) #remove solution index and frontier columns

In [4]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [5]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the sols dataframe

objSenses = {}

In [6]:
# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0 # all objectives are min for curr example
objSenses["Northern Spotted Owl Habitat (ha)"] = 1 #except owl

In [7]:
# Get the objectives' bounds

objBounds = {}
for obj,sense in objSenses.items():
    objBounds[obj] = [sols[obj].min(),sols[obj].max()]

In [8]:
# Get each objective's ideal value

idealObjVals = {}
for obj,bounds in objBounds.items():
    idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max

In [9]:
# Get each objective's worst value

worstObjVals = {}
for obj,bounds in objBounds.items():
    worstObjVals[obj] = bounds[not(objSenses[obj])]

In [10]:
# Compute the non-normalized volume of the objective space

objSpaceVolume_nonNorm = 1
for obj,bounds in objBounds.items():
    objSpaceVolume_nonNorm *= bounds[1] - bounds[0]

In [11]:
# Normalize the objective space by converting each value to the relative achievement along its axis:

#   distance from the worst case value
# ---------------------------------------
# total distance spanned by the objective

# Objective space is now in [0,1]^N where N is number of objectives


for obj,bounds in objBounds.items():
    sols[obj] = sols[obj].apply(
        lambda x: \
        abs(x - worstObjVals[obj]) / \
        (bounds[1]-bounds[0]))

### Algorithm to compute volume of frontier

#### "Add-the-sides" frontier volume computation method

For a mathematical description of the below algorithm, see ./algorithmWriteUp/frontierVolumeComputation_writeup.pdf

In [24]:
# Initialize frontier volume

frontierVolume = 0

In [25]:
# Initialize sub-dimensional (N-1) volume

subDimVolume = 0

In [26]:
# Initialize the dictionary of solutions' sub-dimensional (N-1) volume contributions
# index: contribution 

subDimContributions = {}

In [27]:
# Initialize the array of boundary solutions - those not dominated in the sub-dimensional space

subDimBoundarySols = []

In [28]:
# Sort the dataframe in descending order from first column
# (always descending, since objectives normalized bad->good = 0->1)

sols = sols.sort_values(by=[objs[0]],ascending=False)

In [29]:
# Get list of all non-primary dimensions

dims_secondary = [col for col in sols.columns if col not in [objs[0]]]

In [30]:
# For measuring time
start = time.clock()

# Compute the volume

# For each solution...
for idx,row in sols.iterrows():
    
    # get its raw sub-dimensional volume contribution, which currently includes overlap
    
    subDimContribution = row[dims_secondary].product()
    
    
    # Subtract away the contributions from all previous solutions
    
    for sol,subDimContrib in subDimContributions.items():
        subDimContribution -= subDimContrib
        
        
    # Add back the sub-dimensional volume that lies outside the current solution
    
    ## First, get the current list of boundary solutions.
    ## This is all previous boundary solutions, plus the current solution, minus any that the current solution dominates
    subDimBoundarySols.append(idx)
    enclosedSols = [sol for sol in subDimBoundarySols if (sols[dims_secondary].ix[sol] < row[dims_secondary]).all()]
    for sol in enclosedSols:
        subDimBoundarySols.remove(sol)
    
    ## Next, iterate over the sub-dimensions to get the volume of the sides along each axis
    for dim in dims_secondary:
        ## Get ordered list of boundary solutions that have component along dim-axis that is greater than the current solution
        dimSideSols = sols.ix[subDimBoundarySols]
        dimSideSols = dimSideSols.loc[dimSideSols[dim] > row[dim]]
        ## If there are no better soltions along this axis, then we are done
        if dimSideSols.empty:
            continue
        ## Otherwise, we have sub-dimensional volume to add back.
        ## Get the list of other sub-dimensions, needed to compute side-solution sub-dimensional volume contributions
        otherSecondaryDims = [d for d in dims_secondary if d != dim]
        ## Append the current solution to the list of side solutions
        dimSideSols = dimSideSols.append(sols.ix[idx])
        ## Sort the list in ascending order along the dim-axis
        dimSideSols = dimSideSols.sort_values(by=[dim], ascending=True)
        ## Iterate over side solutions and get the subdim volume contributions
        ### but first a couple loop primers:
        prevDimComponent = 0
        solDimComponent = 0
        for sideSolIdx,sideSolRow in dimSideSols.iterrows():
            ## performing operations on pairs of side solutions, so skip the first
            ## the first solution will always be the one for which we are seeking the volume contribution
            ## because we built the set to be ascending starting from its component along this dimension
            if sideSolIdx == idx:
                prevDimComponent = sideSolRow[dim]
                continue
            solDimComponent = sideSolRow[dim]
            dimDelta = solDimComponent - prevDimComponent
            subDimContribution += dimDelta*sideSolRow[otherSecondaryDims].product()
            prevDimComponent = solDimComponent
        
        
            
    # Updating data objects
    
    subDimContributions[idx] = subDimContribution
    subDimVolume += subDimContribution 
    frontierVolume += subDimContribution*row[objs[0]]

end = time.clock()

In [31]:
print ("Elapsed time: " + (str(end-start)))
frontierVolume

Elapsed time: 1.846134371449125


0.74655676448721253